In [39]:
import pandas as pd
import numpy as np
from datetime import date
import datetime

df = pd.read_csv('data/final_date.csv')

class EventAnalyzer:
    def __init__(self, df):
        self.df = df

#------------------------------------------------------------
# private method
#------------------------------------------------------------

    def _get_combined_dates(self, event_names):
        # 여러 이벤트 이름에 해당하는 날짜들을 하나의 리스트로 합쳐 반환합니다.
        combined_dates = []
        for event_name in event_names:
            row = self.df[self.df['event_name'] == event_name].iloc[0, 1:].dropna()
            dates = [datetime.datetime.strptime(str(date).replace('.', '-'), '%Y-%m-%d').date() for date in row]
            combined_dates.extend(dates)
        
        combined_dates.sort()
        return combined_dates
    
#------------------------------------------------------------
# public method
#------------------------------------------------------------

    def event_intervals(self, event_names, start_year=None, end_year=2023):
        # 합쳐진 여러 이벤트 이름에 대해 연속된 날짜 간의 간격을 반환합니다.
        dates = self._get_combined_dates(event_names)
        if start_year:
            dates = [date for date in dates if start_year <= date.year <= end_year]
        intervals = [(dates[i+1] - dates[i]).days for i in range(len(dates)-1)]
        return intervals

    def event_counts_by_year(self, event_names, start_year=None, end_year=2023):
        # 합쳐진 여러 이벤트 이름에 대해 연도별 발생 횟수를 반환합니다.
        dates = self._get_combined_dates(event_names)
        if start_year:
            dates = [date for date in dates if start_year <= date.year <= end_year]
        counts = {}
        for date in dates:
            year = date.year
            if year in counts:
                counts[year] += 1
            else:
                counts[year] = 1
        return counts

    def get_event_intervals_by_year(self, event_name, year, start_year=None, end_year=2023):
        # 주어진 event_name과 year 정보를 바탕으로 해당 year에 실행된 date 간의 간격을 반환합니다.
        dates = self._get_combined_dates([event_name])
        if start_year:
            dates = [date for date in dates if start_year <= date.year <= end_year]
        dates_of_year = [date for date in dates if date.year == year]
        intervals = [(dates_of_year[i+1] - dates_of_year[i]).days for i in range(len(dates_of_year)-1)]
        return intervals

    def get_event_summary(self, start_year=2017, end_year=2023):
        # 각 이벤트의 주어진 연도 범위 내에서의 발생 횟수, 간격의 평균 및 표준편차 정보를 담은 데이터 프레임을 반환합니다.
        years = list(range(start_year, end_year+1))
        event_names = self.df['event_name'].tolist()
        
        data = []
        for event_name in event_names:
            yearly_counts = []
            yearly_avg_intervals = []
            yearly_std_intervals = []
            for year in years:
                yearly_counts.append(self.event_counts_by_year([event_name], start_year, end_year).get(year, 0))
                yearly_intervals = self.get_event_intervals_by_year(event_name, year, start_year, end_year)
                yearly_avg_intervals.append(np.mean(yearly_intervals) if yearly_intervals else None)
                yearly_std_intervals.append(np.std(yearly_intervals) if yearly_intervals else None)
            
            overall_intervals = self.event_intervals([event_name], start_year, end_year)
            overall_avg_interval = np.mean(overall_intervals) if overall_intervals else None
            overall_std_interval = np.std(overall_intervals) if overall_intervals else None

            data_row = yearly_counts + yearly_avg_intervals + yearly_std_intervals + [overall_avg_interval, overall_std_interval]
            data.append(data_row)
        
        columns = (
            [f"Count_{year}" for year in years] +
            [f"Avg_Interval_{year}" for year in years] +
            [f"Std_Interval_{year}" for year in years] +
            ["Overall_Avg_Interval", "Overall_Std_Interval"]
        )
        summary_df = pd.DataFrame(data, columns=columns, index=event_names)
        
        return summary_df

# 테스트
analyzer = EventAnalyzer(df)
event_summary_result = analyzer.get_event_summary(start_year=2021)
event_summary_result

,Count_2021,Count_2022,Count_2023,Avg_Interval_2021,Avg_Interval_2022,Avg_Interval_2023,Std_Interval_2021,Std_Interval_2022,Std_Interval_2023,Overall_Avg_Interval,Overall_Std_Interval
10성 이하에서 스타포스 강화 성공 시 1+1 강화,6,4,4,51.800,72.333333,54.666667,5.600000,11.897712,17.249799,65.692308,28.778690
룬 경험치 버프 효과 +100%,11,11,6,31.500,32.900000,28.000000,8.991663,11.734138,8.854377,31.888889,10.140982
"몬스터 컬렉션 '의문의 모몽' 지급, 신규 몬스터 등록 확률 추가",5,4,2,68.250,81.666667,84.000000,47.314770,47.590849,0.000000,81.200000,41.906563
몬스터 파크 클리어 경험치 추가 50%,6,7,7,56.000,50.166667,33.833333,27.647785,33.672029,13.655483,48.263158,28.265363
미라클 타임,1,2,2,NaN,189.000000,182.000000,NaN,0.000000,0.000000,227.500000,74.821454
소울 조각 사용 시 위대한 소울 획득 확률 5배,5,5,4,56.000,66.500000,63.666667,4.949747,31.500000,5.792716,63.538462,20.144155
"스타포스 5, 10, 15성에서 강화 시도 시 성공확률 100%",7,7,4,56.000,52.500000,56.333333,11.430952,24.500000,28.170118,53.529412,21.180228
스타포스 강화 비용 30% 할인,6,6,5,53.200,57.400000,42.250000,9.495262,16.800000,30.384001,55.125000,21.937055
어빌리티 재설정 비용 50% 할인,7,7,5,52.500,56.000000,43.250000,31.500000,15.121728,26.611792,50.555556,24.748114
"폴로와 프리토, 에스페시아의 현상금 사냥 경험치, 불꽃늑대 퇴장 시 획득하는 경험치 추가",9,10,6,30.625,37.333333,32.200000,6.945097,22.622506,15.078461,34.708333,16.843592


In [22]:
events_name = list(analyzer.data_count().keys())

for i in range(len(events_name)):
    for j in range(len(events_name)):
        if i == j:
            continue
        common_dates = analyzer.get_common_dates(events_name[i], events_name[j])
        ratio = len(common_dates) / min(analyzer.data_count()[events_name[i]], analyzer.data_count()[events_name[j]])
        if ratio > 0.2:
            print(events_name[i],',' ,events_name[j],':' ,ratio, (analyzer.get_common_dates(events_name[i], events_name[j])))



룬 경험치 버프 효과 +100% , 몬스터 파크 클리어 경험치 추가 50% : 0.20689655172413793 ['2021-09-12', '2021-10-31', '2022-01-30', '2022-04-24', '2022-09-25', '2022-10-09']
몬스터 파크 클리어 경험치 추가 50% , 룬 경험치 버프 효과 +100% : 0.20689655172413793 ['2021-09-12', '2021-10-31', '2022-01-30', '2022-04-24', '2022-09-25', '2022-10-09']
몬스터 파크 클리어 경험치 추가 50% , 폴로와 프리토, 에스페시아의 현상금 사냥 경험치, 불꽃늑대 퇴장 시 획득하는 경험치 추가 : 0.2413793103448276 ['2021-05-02', '2022-08-07', '2022-09-25', '2023-01-08', '2023-02-19', '2023-04-16', '2023-04-30']
소울 조각 사용 시 위대한 소울 획득 확률 5배 , 어빌리티 재설정 비용 50% 할인 : 0.2857142857142857 ['2018-09-09', '2018-12-30', '2019-03-17', '2019-07-28', '2019-09-15', '2019-12-15', '2020-02-09', '2021-10-10', '2022-09-25', '2023-05-14']
어빌리티 재설정 비용 50% 할인 , 소울 조각 사용 시 위대한 소울 획득 확률 5배 : 0.2857142857142857 ['2018-09-09', '2018-12-30', '2019-03-17', '2019-07-28', '2019-09-15', '2019-12-15', '2020-02-09', '2021-10-10', '2022-09-25', '2023-05-14']
폴로와 프리토, 에스페시아의 현상금 사냥 경험치, 불꽃늑대 퇴장 시 획득하는 경험치 추가 , 몬스터 파크 클리어 경험치 추가 50% : 0.2413793103

In [30]:
analyzer.event_intervals(['어빌리티 재설정 비용 50% 할인'])

[42,
 56,
 91,
 196,
 133,
 70,
 42,
 77,
 42,
 49,
 42,
 49,
 91,
 56,
 35,
 84,
 35,
 35,
 63,
 49,
 35,
 119,
 56,
 21,
 42,
 35,
 42,
 56,
 77,
 70,
 42,
 63,
 35,
 49,
 30,
 5,
 77,
 35,
 56]

In [32]:
analyzer.get_event_summary()

AttributeError: 'EventAnalyzer' object has no attribute 'get_event_summary'